In [3]:
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import PySpark Pandas
# import pyspark.pandas as ps

In [4]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Users\\devenv\\SPARK\\spark-3.0.0-preview2-bin-hadoop2.7'

In [5]:
# os.chdir(r'C:\Users\devenv\SPARK\spark-3.0.0-preview2-bin-hadoop2.7')
pyspark.__version__

'2.4.5'

In [5]:
# this bit is in both the medium article and learning spark
from pyspark import SparkContext, SparkConf

In [ ]:
# this bit is just in the medium article
from pyspark.sql import SparkSession

In [ ]:
# conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
# see https://spark.apache.org/docs/latest/configuration.html#application-properties for app properties
# app name is defined as The name of your application. This will appear in the UI and in log data. 
conf = SparkConf().setAppName('appName').setMaster('local')
# setMaster(value) − To set the master URL
# Master could be "local[2]" and set up multple "threads" or look like .setMaster("spark://master:7077")
# you can also set the config blank and pass conf parameters at run time e.g. via the command line
# this example is given
# ./bin/spark-submit --name "My app" --master local[4] --conf spark.eventLog.enabled=false myApp.py

# SparkConf() will load the values from spark.* Java system properties as well as what we pass

In [ ]:
# the book
sc = SparkContext(conf=conf)
# this just medium
# sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc) # we seem to need this or we error

In [ ]:
# sc.stop()

In [ ]:
nums = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
type(nums)

In [ ]:
nums.take(3)

In [ ]:
nums.map(lambda x: x * x).collect()

In [ ]:
num1 = nums.map(lambda x: [y for y in range(x * x)])

In [ ]:
num1, type(num1)

In [ ]:
num1.collect()

In [ ]:
gb = nums.groupBy(lambda x : x % 2 == 0).collect()

In [ ]:
gb

In [ ]:
group_object = num1.groupBy(lambda x : int(len(x)/10)).collect()

In [ ]:
group_object

In [ ]:
firstGroup = group_object[0]
[x for x in firstGroup]

In [ ]:
[x for x in firstGroup[1]]

In [ ]:
nums.name()

In [ ]:
lines = sc.textFile('C:\Shared Files\example_data_(002)\Questions.csv')

In [ ]:
type(lines)

In [ ]:
lines.count(), lines.first()

In [ ]:
type(lines)

In [ ]:
lines

In [ ]:
pylines = lines.filter(lambda x: 'leader' in x)

In [ ]:
pylines, pylines.count()

In [ ]:
type(pylines)

In [ ]:
pylines.first(), type(pylines.first())

In [ ]:
pylines.first().split(',')[1]

In [ ]:
pylines.take(1) # returns a list

In [ ]:
pylines.take(1)[0].split(',')

In [ ]:
help(lines.map)

In [ ]:
lines.map(lambda x: (x, 1)).take(5)

In [ ]:
lines.map(lambda x: (x, 1)).take(5)[0],\
type(lines.map(lambda x: (x, 1)).take(5)[0]),\
type(lines.map(lambda x: (x, 1)).take(1))

In [ ]:
type(lines.map(lambda x: (x, 1))), type(lines)

In [ ]:
help(pyspark.rdd.PipelinedRDD)

In [ ]:
type(lines.foreach(lambda x : x + 'dude'))

In [ ]:
lines.collect()

In [ ]:
lout = [x for x in range(lines.count())]

In [ ]:
lout

In [ ]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
agg_tst = sc.parallelize([1, 2, 3, 4])
agg_tst_2 = sc.parallelize([1, 2, 3, 4], 2)

In [ ]:
agg_tst.collect(), agg_tst_2.collect()

In [ ]:
agg_tst.glom().collect(), agg_tst_2.glom().collect()

In [ ]:
agg_tst.aggregate((0, 0), seqOp, combOp)
#      (10, 4)
#  sc.parallelize([]).aggregate((0, 0), seqOp, combOp)
#    (0, 0)

In [ ]:
help(lines.glom)

In [ ]:
agg_tst.collect()

In [ ]:
agg_tst_2.collect(), agg_tst_2.glom().collect()

#### function testing

In [ ]:
a = sc.parallelize([1,2,3,4], 2)

In [ ]:
type(a)

In [ ]:
rspDf = spark.read.option("header", True).csv(r"C:\Shared Files\example_data_(002)\ResponseData.csv")

In [ ]:
rspDf.take(2)

In [ ]:
type(rspDf)

In [ ]:
smmry = rspDf.summary().collect()
smmry

In [ ]:
pd.DataFrame(smmry) # hmm no headers...

In [ ]:
# pass to the list to spark df 
smmrySDf = spark.createDataFrame(smmry)
smmrySDf.show()

In [ ]:
# then pandas df
smmryPDf = smmrySDf.toPandas()
smmryPDf

In [ ]:
rspDf.columns

In [ ]:
rspDf.stat.freqItems(['Response'], support=.99).collect()

In [ ]:
rspDf.Response

In [ ]:
rspDf.Response.contains('1')

In [ ]:
rspDf.select(rspDf.Response.contains('1')).show(5)

In [ ]:
rspDf.show(5)

In [ ]:
rspDf_int_resp = rspDf.withColumn('Response', rspDf.Response.cast('int'))
rspDf_int_resp.groupby('QuestionID').sum('Response').show(5)

In [ ]:
rspDfgp =(
rspDf  # rspDf_int_resp
.groupby(['QuestionID','ResponseID'])
.agg({'Response':'count'})  # .count()  # .max('Response')
.sort('QuestionID')
.filter(rspDf.QuestionID.isin(['1014663','1014649']))
) 
rspDfgp.show(5)  

In [ ]:
rspDfgp.filter(rspDfgp['count(Response)'] > 1).select('ResponseID').show()  #.select(rspDfgp.ResponseID).show()

In [ ]:
qrDf = rspDf.groupby(['QuestionID', 'Response']).count() # get a pivot of question ids and responses
qrDf.sort('count', ascending=False).show(n=5)

In [ ]:
# maybe simpler in sql
try:
    rspDf.createTempView("rspsql")
except Exception as e:
    print(e)
    pass

In [ ]:
spark.sql(
'''select QuestionID,
ResponseID,
count(Response)
from rspsql
where QuestionID
in (\'1014663\',\'1014649\')
group by QuestionID, ResponseID
having count(Response) >1
''').show()

In [ ]:
# we can do correlared subqueries
spark.sql(
    '''select distinct QuestionID,
    (select max(a.Response) from rspsql a where a.QuestionID = b.QuestionID and response <> 5)
    from rspsql b''').show(10)

In [ ]:
# returns a useable pyspark dataframe
type(spark.sql(
    '''select distinct QuestionID,
    (select max(a.Response) from rspsql a where a.QuestionID = b.QuestionID and response <> 5)
    from rspsql b''')
    )

In [ ]:
import pyspark.sql.functions as sf

In [ ]:
top5qrDf = qrDf.groupby(['QuestionID']).agg(sf.sum('count').alias('nrResponses')) \
                                            .sort('nrResponses', ascending=False).take(5)
print(top5qrDf, '\n')
top5QLs = [x.QuestionID for x in top5qrDf]
print(top5QLs, '\n')
top5Qs = qrDf.filter(qrDf.QuestionID.isin(top5QLs)).sort('QuestionID')
top5Qs.show()

In [ ]:
# how about we add a column to top5Qs to show the total responses for that question
QuestionResponses = qrDf.groupby(['QuestionID']).agg(sf.sum('count').alias('nrResponses')) \
                                        .sort('nrResponses', ascending=False)
print(QuestionResponses.head(2),'\n')
thig = top5Qs.join(QuestionResponses, on='QuestionID', how='inner')
thig.show(n=100)

In [ ]:
print(rspDf.rdd.getNumPartitions())
rep_rspDf = rspDf.repartition(8)
rspDf.rdd.getNumPartitions(), rep_rspDf.rdd.getNumPartitions()

In [ ]:
# mapPartitons & mapPartitionsWithIndex - pass the partition to the function with values as an iteraror.
# This is more efficient as instances (i.e functions) only have to be created once per partition not for each element

tst = sc.parallelize([[1, 2], [2, 3], [3, 4], [4, 5]], 4)
def f(ix, val): yield (ix, type(val))
tst.mapPartitionsWithIndex(f).collect()  # WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): yield (ix, [x for x in val][0])
tst.mapPartitionsWithIndex(f).collect()  # WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): return (ix, type(val))
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): return (ix, [x for x  in val][0])
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): return (ix, *val)
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): yield (ix, *val)
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): yield [x for x in val][0]
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
tst = sc.parallelize([[1,2], [2,3], [3,4], [4,5]], 4)
def f(ix, val): return [x for x in val][0]
tst.mapPartitionsWithIndex(f).collect() #WithIndex

In [ ]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]
dfNms = spark.createDataFrame(data=arrayData, schema=['name','knownLanguages','properties'])
dfNms.show(truncate=False)

In [ ]:
from pyspark.sql.functions import explode
dfNms2 = dfNms.select(dfNms.name, explode(dfNms.knownLanguages).alias('knownLanguages'), dfNms.properties)
dfNms2.printSchema()
dfNms2.show()

In [ ]:
dfNms3 = dfNms2.select(dfNms2.name, dfNms2.knownLanguages, explode(dfNms2.properties))
dfNms3.printSchema()
dfNms3.show()

#### More Testing out Pyspark DataFrames

In [ ]:
respdf1 = spark.read.option("header", True).csv(r"C:\Shared Files\example_data_(002)\ResponseData.csv")

In [ ]:
type(respDf1)

In [ ]:
respdf1.show(5)

In [ ]:
qdf1 = spark.read.option("header", True).csv('C:\Shared Files\example_data_(002)\Questions.csv')

In [ ]:
qdf1.show(5)

In [ ]:
(qdf1.count(), len(qdf1.columns)) # gets us the shape

In [ ]:
qdf1.describe().show()

In [ ]:
qdf1.summary().show()